In [2]:
#Import Libraries

import os
import glob
import datetime

# keras imports
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from keras.models import Model

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import h5py
import json
import time

In [3]:
config={
  "model"           : "vgg16",
  "weights"         : "imagenet",

  "train_path"      : "C:/Users/Eisu/Desktop/data2a/training",
  "test_path"       : "C:/Users/Eisu/Desktop/data2a/validation",
  "features_path"   : "C:/Users/Eisu/Desktop/cdd/car_damage_check/FRS/features.h5",
  "labels_path"     : "C:/Users/Eisu/Desktop/cdd/car_damage_check/FRS/labels.h5",
  "classifier_path" : "C:/Users/Eisu/Desktop/cdd/car_damage_check/FRS/classifier.pickle",
  "model_path"      : "C:/Users/Eisu/Desktop/cdd/car_damage_check/FRS/model",

  "test_size"       : 0.20,
  "seed"            : 9,
}

In [4]:
# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [5]:
# config variables
model_name    = config["model"]
weights     = config["weights"]
train_path    = config["train_path"]
features_path   = config["features_path"]
labels_path   = config["labels_path"]
test_size     = config["test_size"]
model_path    = config["model_path"]

In [6]:
# Here we are loading the base VGG16 model with weights and then excluding the top dense layer
if model_name == "vgg16":
    base_model = VGG16(weights=weights)
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    image_size = (224, 224)
else:
    base_model = None

print ("Successfully loaded base model and model...")

Successfully loaded base model and model...


In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
# path to training dataset
train_labels = os.listdir(train_path)

# encode the labels
print ("Encoding labels...")
le = LabelEncoder()
le.fit([tl for tl in train_labels])

# variables to hold features and labels
features = []
labels   = []

# loop over all the labels in the folder
count = 1
for i, label in enumerate(train_labels):
    cur_path = train_path + "/" + label
    count = 1
    for image_path in glob.glob(cur_path + "/*.jpg"):
        img = image.load_img(image_path, target_size=image_size)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = model.predict(x)
        flat = feature.flatten()
        features.append(flat)
        labels.append(label)
        print ("Processed - " + str(count))
        count += 1
    print ("Completed label - " + label)

# encode the labels using LabelEncoder
le = LabelEncoder()
le_labels = le.fit_transform(labels)

# get the shape of training labels
print ("[STATUS] training labels: {}".format(le_labels))
print ("[STATUS] training labels shape: {}".format(le_labels.shape))

Encoding labels...
1/1 [==============================] - 0s 374ms/step
Processed - 1
1/1 [==============================] - 0s 198ms/step
Processed - 2
1/1 [==============================] - 0s 233ms/step
Processed - 3
1/1 [==============================] - 0s 260ms/step
Processed - 4
1/1 [==============================] - 0s 190ms/step
Processed - 5
1/1 [==============================] - 0s 205ms/step
Processed - 6
1/1 [==============================] - 0s 203ms/step
Processed - 7
1/1 [==============================] - 0s 224ms/step
Processed - 8
1/1 [==============================] - 0s 194ms/step
Processed - 9
1/1 [==============================] - 0s 200ms/step
Processed - 10
1/1 [==============================] - 0s 188ms/step
Processed - 11
1/1 [==============================] - 0s 224ms/step
Processed - 12
1/1 [==============================] - 0s 199ms/step
Processed - 13
1/1 [==============================] - 0s 255ms/step
Processed - 14
1/1 [==============================] -

Processed - 239
1/1 [==============================] - 0s 240ms/step
Processed - 240
1/1 [==============================] - 0s 217ms/step
Processed - 241
1/1 [==============================] - 0s 248ms/step
Processed - 242
1/1 [==============================] - 0s 258ms/step
Processed - 243
1/1 [==============================] - 0s 220ms/step
Processed - 244
1/1 [==============================] - 0s 200ms/step
Processed - 245
1/1 [==============================] - 0s 246ms/step
Processed - 246
1/1 [==============================] - 0s 256ms/step
Processed - 247
1/1 [==============================] - 0s 292ms/step
Processed - 248
1/1 [==============================] - 0s 281ms/step
Processed - 249
1/1 [==============================] - 0s 260ms/step
Processed - 250
1/1 [==============================] - 0s 273ms/step
Processed - 251
1/1 [==============================] - 0s 266ms/step
Processed - 252
1/1 [==============================] - 0s 203ms/step
Processed - 253
1/1 [=============

1/1 [==============================] - 0s 249ms/step
Processed - 58
1/1 [==============================] - 0s 314ms/step
Processed - 59
1/1 [==============================] - 0s 278ms/step
Processed - 60
1/1 [==============================] - 0s 378ms/step
Processed - 61
1/1 [==============================] - 0s 326ms/step
Processed - 62
1/1 [==============================] - 0s 318ms/step
Processed - 63
1/1 [==============================] - 0s 331ms/step
Processed - 64
1/1 [==============================] - 0s 325ms/step
Processed - 65
1/1 [==============================] - 0s 334ms/step
Processed - 66
1/1 [==============================] - 0s 404ms/step
Processed - 67
1/1 [==============================] - 0s 321ms/step
Processed - 68
1/1 [==============================] - 0s 299ms/step
Processed - 69
1/1 [==============================] - 0s 300ms/step
Processed - 70
1/1 [==============================] - 0s 276ms/step
Processed - 71
1/1 [==============================] - 0s 271ms/

1/1 [==============================] - 0s 287ms/step
Processed - 7
1/1 [==============================] - 0s 216ms/step
Processed - 8
1/1 [==============================] - 0s 284ms/step
Processed - 9
1/1 [==============================] - 0s 209ms/step
Processed - 10
1/1 [==============================] - 0s 198ms/step
Processed - 11
1/1 [==============================] - 0s 256ms/step
Processed - 12
1/1 [==============================] - 0s 248ms/step
Processed - 13
1/1 [==============================] - 0s 251ms/step
Processed - 14
1/1 [==============================] - 0s 349ms/step
Processed - 15
1/1 [==============================] - 0s 248ms/step
Processed - 16
1/1 [==============================] - 0s 259ms/step
Processed - 17
1/1 [==============================] - 0s 244ms/step
Processed - 18
1/1 [==============================] - 0s 253ms/step
Processed - 19
1/1 [==============================] - 0s 213ms/step
Processed - 20
1/1 [==============================] - 0s 240ms/ste

Processed - 245
1/1 [==============================] - 0s 212ms/step
Processed - 246
1/1 [==============================] - 0s 232ms/step
Processed - 247
1/1 [==============================] - 0s 231ms/step
Processed - 248
1/1 [==============================] - 0s 223ms/step
Processed - 249
1/1 [==============================] - 0s 237ms/step
Processed - 250
1/1 [==============================] - 0s 230ms/step
Processed - 251
1/1 [==============================] - 0s 218ms/step
Processed - 252
1/1 [==============================] - 0s 241ms/step
Processed - 253
1/1 [==============================] - 0s 254ms/step
Processed - 254
1/1 [==============================] - 0s 238ms/step
Processed - 255
1/1 [==============================] - 0s 220ms/step
Processed - 256
1/1 [==============================] - 0s 235ms/step
Processed - 257
1/1 [==============================] - 0s 232ms/step
Processed - 258
1/1 [==============================] - 0s 249ms/step
Processed - 259
1/1 [=============

In [9]:
# save features and labels
h5f_data = h5py.File(features_path, 'w')
h5f_data.create_dataset('dataset_1', data=np.array(features))

h5f_label = h5py.File(labels_path, 'w')
h5f_label.create_dataset('dataset_1', data=np.array(le_labels))

h5f_data.close()
h5f_label.close()

# save model and weights
model_json = model.to_json()
with open(model_path + str(test_size) + ".json", "w") as json_file:
    json_file.write(model_json)

# save weights
model.save_weights(model_path + str(test_size) + ".h5")
print("Saved model and weights to disk..")

print ("Features and labels saved..")

# end time
end = time.time()
print ("End time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))

Saved model and weights to disk..
Features and labels saved..
End time - 2022-12-14 19:43
